In [1]:
%run ./apiMethods.ipynb
%run ./arenaCrawlers.ipynb

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.01 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 4.77 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.01 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.01 µs
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 5.96 µs
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.25 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 13.1 µs


In [2]:
def createAllTMsJSONandId(allTms):
    allTmJsonsWithid = []
    for machine in allTms:
        index = ((int(machine)-1)*18) + 1
        allTmJsonsWithid.append((requestMachine(index), machine))
    return allTmJsonsWithid


In [3]:
def createAllMoveJSONfromTMJSON(allTmJson):
    allMovesByTmJsons = []
    for tmJson in allTmJson:
        allMovesByTmJsons.append( (requestMove(tmJson[0]['move']['name']), tmJson[1] ) )
    return allMovesByTmJsons

In [8]:
def createIdToMoveNameAndAllowedPokemon(allMoveJsonAndId, encounterablePokemonTable):
    idObject = {}
    for moveJson in allMoveJsonAndId:
        tmId = moveJson[1]
        moveJson = moveJson[0]
        moveName = moveJson['name']
        allowedPoke = []
        for pokemon in moveJson['learned_by_pokemon']:
            name = pokemon['name']
            if(encounterablePokemonTable.name.str.contains(name).any()):
                allowedPoke.append(name)
        idObject[tmId] = {"name":moveName,'allowedPoke':allowedPoke} 
    return idObject
                 #  pokemonToTM[name].append(moveName)

In [5]:
def appendTmMovesToMovesOfInterestCSV(moveJsonsWithId, moveTable):
    movesToAppend = []
    movesOfInterestColumNames = ['name', 'moveType', 'damageClass', 'power', 'pp', 'priority', 'accuracy']
    
    for moveJsonWithId in moveJsonsWithId:
        moveJson = moveJsonWithId[0]
        name = moveJson['name']
        if name not in moveTable.name:
            # extracts moves data to CSV format
            moveObject = produceMoveWithoutFetch(moveJson)
            moveCSVformat = [moveObject['name'],moveObject['moveType'],moveObject['damageClass'],moveObject['power'],moveObject['pp'],moveObject['priority'],moveObject['accuracy']]
            moveTable = moveTable.append({'name':moveObject['name'], 'moveType':moveObject['moveType'],'damageClass': moveObject['damageClass'],'power': moveObject['power'],'pp': moveObject['pp'], 'priority':moveObject['priority'],'accuracy':moveObject['accuracy']},ignore_index=True)
    return moveTable
            # ----- #
    #TODO WRITE table ToFile 
    #index=False

In [9]:
#allTmJson = createAllTMsJSONandId(POSSIBLE_TM_ARENA9)
#allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

# remove -1 wenn arena 9 gecrawled
for i in range(8, len(machineArrays) ):
    arrayIndex = i
    arenaIndex = i+1
    print('ARENA ' , arenaIndex)
    pathMoves = f'Data/red/arena-{arenaIndex}/MovesOfInterest.csv'
    pathPokemon = f'Data/red/arena-{arenaIndex}/EncounterablePokemon.csv'
    movesOfInterestTable = pd.read_csv(pathMoves)
    movesOfInterestTable.drop(columns=movesOfInterestTable.columns[0], axis=1, inplace=True)
    movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
    appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
    print(movesOfInterestTable.tail())
    
    encounterablePokemonTable = pd.read_csv(pathPokemon)    
    
    pokeToNewMoves = {}
    tmIdToMoveAndAllowedPoke = createIdToMoveNameAndAllowedPokemon(allMoveJson, encounterablePokemonTable)
    for tmId in machineArrays[arrayIndex]:
        tmIdNumber = tmId
        allowedPoke = tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']
        moveName = tmIdToMoveAndAllowedPoke[tmIdNumber]['name']
        for pokeName in tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']:
            if pokeName in pokeToNewMoves.keys():
                pokeToNewMoves[pokeName].append(moveName)
            else:
                 pokeToNewMoves[pokeName] = [moveName]
    newMoveArrays = []
    for name in encounterablePokemonTable.name:
        if name in pokeToNewMoves.keys():
            newMoveArrays.append(pokeToNewMoves[name])
        else:
            newMoveArrays.append([])
    encounterablePokemonTable['movesByTM'] = newMoveArrays
    
    
    encounterablePokemonTable.to_csv(pathPokemon,index=False) 
    movesOfInterestTable.to_csv(pathMoves,index=False) 


ARENA  9
           name moveType damageClass power  pp  priority accuracy
186   metronome   normal      status  None  10         0     None
187  fire-blast     fire     special   110   5         0       85
188     fissure   ground    physical  None   5         0       30
189  sky-attack   flying    physical   140   5         0       90
190   explosion   normal    physical   250   5         0      100


In [275]:
idToMoveAndPokemon = createIdToMoveNameAndAllowedPokemon(allMoveJson, 1)

In [11]:
def writeTMtoCSV(game):
    #allTmJson = createAllTMsJSONandId(POSSIBLE_TM_ARENA9)
    #allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

    for i in range(0, len(machineArrays) ):
        arrayIndex = i
        arenaIndex = i+1
        print('ARENA ' , arenaIndex)
        pathMoves = f'Data/{game}/arena-{arenaIndex}/MovesOfInterest.csv'
        pathPokemon = f'Data/{game}/arena-{arenaIndex}/EncounterablePokemon.csv'
        movesOfInterestTable = pd.read_csv(pathMoves)
        movesOfInterestTable.drop(columns=movesOfInterestTable.columns[0], axis=1, inplace=True)
        movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        print(movesOfInterestTable.tail())
        
        encounterablePokemonTable = pd.read_csv(pathPokemon)    
        
        pokeToNewMoves = {}
        tmIdToMoveAndAllowedPoke = createIdToMoveNameAndAllowedPokemon(allMoveJson, encounterablePokemonTable)
        for tmId in machineArrays[arrayIndex]:
            tmIdNumber = tmId
            allowedPoke = tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']
            moveName = tmIdToMoveAndAllowedPoke[tmIdNumber]['name']
            for pokeName in tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']:
                if pokeName in pokeToNewMoves.keys():
                    pokeToNewMoves[pokeName].append(moveName)
                else:
                     pokeToNewMoves[pokeName] = [moveName]
        newMoveArrays = []
        for name in encounterablePokemonTable.name:
            if name in pokeToNewMoves.keys():
                newMoveArrays.append(pokeToNewMoves[name])
            else:
                newMoveArrays.append([])
        encounterablePokemonTable['movesByTM'] = newMoveArrays
        
        
        encounterablePokemonTable.to_csv(pathPokemon,index=False) 
        movesOfInterestTable.to_csv(pathMoves,index=False) 

In [12]:
writeTMtoCSV('blue')

ARENA  1
           name moveType damageClass power  pp  priority accuracy
104   metronome   normal      status  None  10         0     None
105  fire-blast     fire     special   110   5         0       85
106     fissure   ground    physical  None   5         0       30
107  sky-attack   flying    physical   140   5         0       90
108   explosion   normal    physical   250   5         0      100
ARENA  2
           name moveType damageClass power  pp  priority accuracy
137   metronome   normal      status  None  10         0     None
138  fire-blast     fire     special   110   5         0       85
139     fissure   ground    physical  None   5         0       30
140  sky-attack   flying    physical   140   5         0       90
141   explosion   normal    physical   250   5         0      100
ARENA  3
           name moveType damageClass power  pp  priority accuracy
143   metronome   normal      status  None  10         0     None
144  fire-blast     fire     special   110   5   